집계와 조인
이 교시를 마치면 다음을 할 수 있습니다:

groupBy()와 agg()로 그룹별 집계를 수행할 수 있다
count(), sum(), avg(), min(), max() 집계 함수를 활용할 수 있다
다양한 조인 타입(inner, left, right, outer)을 이해하고 적용할 수 있다
pivot()으로 데이터를 피벗 테이블 형태로 변환할 수 있다
Spark SQL을 DataFrame과 함께 활용할 수 있다

In [1]:
# -----------------------------------------------------------------------------
# 환경 설정: SparkSession 및 테스트 데이터 준비
# -----------------------------------------------------------------------------
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, lit, when, count, sum, avg, min, max,
    countDistinct, first, last, collect_list, collect_set,
    round as spark_round, expr
)
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import pandas as pd
import numpy as np
import os

# SparkSession 생성
spark = SparkSession.builder \
    .appName("PySpark-Aggregation-Join") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", 10) \
    .getOrCreate()

# 테스트 데이터 디렉토리
os.makedirs("/tmp/spark_tutorial", exist_ok=True)

np.random.seed(42)

# -----------------------------------------------------------------------------
# 테스트 데이터 1: 직원 정보
# -----------------------------------------------------------------------------
employees = pd.DataFrame({
    "emp_id": [f"E{i:03d}" for i in range(1, 51)],
    "name": [f"Employee_{i}" for i in range(1, 51)],
    "department": np.random.choice(
        ["Engineering", "Sales", "Marketing", "HR", "Finance"], 50
    ),
    "salary": np.random.randint(40000, 120000, 50),
    "age": np.random.randint(25, 55, 50),
    "hire_year": np.random.choice([2020, 2021, 2022, 2023, 2024], 50),
})
employees.to_csv("/tmp/spark_tutorial/employees.csv", index=False)

# -----------------------------------------------------------------------------
# 테스트 데이터 2: 부서 정보 (조인용)
# -----------------------------------------------------------------------------
departments = pd.DataFrame({
    "dept_name": ["Engineering", "Sales", "Marketing", "HR", "Finance", "Legal"],
    "dept_head": ["Alice", "Bob", "Charlie", "Diana", "Eve", "Frank"],
    "budget": [500000, 300000, 200000, 150000, 400000, 100000],
    "location": ["Seoul", "Busan", "Seoul", "Daegu", "Seoul", "Incheon"],
})
departments.to_csv("/tmp/spark_tutorial/departments.csv", index=False)

# -----------------------------------------------------------------------------
# 테스트 데이터 3: 매출 데이터 (시계열)
# -----------------------------------------------------------------------------
sales = pd.DataFrame({
    "date": pd.date_range("2026-01-01", periods=100, freq="D").strftime("%Y-%m-%d"),
    "product": np.random.choice(["A", "B", "C"], 100),
    "region": np.random.choice(["Seoul", "Busan", "Daegu"], 100),
    "amount": np.random.randint(100, 1000, 100),
    "quantity": np.random.randint(1, 20, 100),
})
sales.to_csv("/tmp/spark_tutorial/sales.csv", index=False)

# DataFrame 로드
df_emp = spark.read.csv("/tmp/spark_tutorial/employees.csv", header=True, inferSchema=True)
df_dept = spark.read.csv("/tmp/spark_tutorial/departments.csv", header=True, inferSchema=True)
df_sales = spark.read.csv("/tmp/spark_tutorial/sales.csv", header=True, inferSchema=True)

print("데이터 로드 완료!")
print(f"직원: {df_emp.count()}명, 부서: {df_dept.count()}개, 매출: {df_sales.count()}건")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/20 06:34:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


데이터 로드 완료!
직원: 50명, 부서: 6개, 매출: 100건


##View(뷰)

In [ ]:
# -----------------------------------------------------------------------------
# createOrReplaceTempView(): SQL 테이블 등록
# -----------------------------------------------------------------------------
#
# createOrReplaceTempView(이름): 임시 뷰로 등록
# - 해당 SparkSession 내에서만 유효
# - 세션 종료 시 자동 삭제
# - 같은 이름의 뷰가 있으면 덮어쓰기 (Replace)

# DataFrame을 SQL 테이블(뷰)로 등록
df_emp.createOrReplaceTempView("employees")
df_dept.createOrReplaceTempView("departments")

print("SQL 테이블 등록 완료: employees, departments")

# -----------------------------------------------------------------------------
# 등록 후 사용 예시
# -----------------------------------------------------------------------------
# 이제 SQL 문법으로 DataFrame을 조회할 수 있습니다!
# spark.sql("SELECT * FROM employees WHERE salary > 5000")
# spark.sql("SELECT dept_id, AVG(salary) FROM employees GROUP BY dept_id")

SQL 테이블 등록 완료: employees, departments


26/01/20 06:34:34 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [3]:
# -----------------------------------------------------------------------------
# spark.sql(): SQL 쿼리 실행
# -----------------------------------------------------------------------------

# spark.sql(쿼리문): SQL 실행 후 DataFrame 반환
# 복잡한 로직을 SQL로 작성 가능

# SQL로 부서별 통계
sql_result = spark.sql("""
    SELECT
        department,
        COUNT(*) as `인원수`,
        ROUND(AVG(salary), 2) as `평균급여`,
        MAX(salary) as `최고급여`
    FROM employees
    GROUP BY department
    ORDER BY `인원수` DESC
""")

print("=== SQL 쿼리 결과 ===")
sql_result.show()

=== SQL 쿼리 결과 ===
+-----------+------+--------+--------+
| department|인원수|평균급여|최고급여|
+-----------+------+--------+--------+
|         HR|    13|85255.23|  112409|
|  Marketing|    10| 76457.5|  117189|
|      Sales|    10| 70509.7|  111211|
|    Finance|    10| 82034.8|  118953|
|Engineering|     7|79437.14|  107563|
+-----------+------+--------+--------+



In [4]:
# -----------------------------------------------------------------------------
# spark.sql(): SQL 쿼리 실행
# -----------------------------------------------------------------------------

# spark.sql(쿼리문): SQL 실행 후 DataFrame 반환
# 복잡한 로직을 SQL로 작성 가능

# SQL로 부서별 통계
sql_result = spark.sql("""
    SELECT
        department,
        COUNT(*) as `인원수`,
        ROUND(AVG(salary), 2) as `평균급여`,
        MAX(salary) as `최고급여`
    FROM employees
    GROUP BY department
    ORDER BY `인원수` DESC
""")

print("=== SQL 쿼리 결과 ===")
sql_result.show()

=== SQL 쿼리 결과 ===
+-----------+------+--------+--------+
| department|인원수|평균급여|최고급여|
+-----------+------+--------+--------+
|         HR|    13|85255.23|  112409|
|  Marketing|    10| 76457.5|  117189|
|      Sales|    10| 70509.7|  111211|
|    Finance|    10| 82034.8|  118953|
|Engineering|     7|79437.14|  107563|
+-----------+------+--------+--------+



In [5]:
# -----------------------------------------------------------------------------
# SQL: 조인
# -----------------------------------------------------------------------------

# SQL로 조인 쿼리
sql_join = spark.sql("""
    SELECT
        e.name,
        e.department,
        e.salary,
        d.dept_head,
        d.location
    FROM employees e
    JOIN departments d ON e.department = d.dept_name
    WHERE e.salary >= 70000
    ORDER BY e.salary DESC
""")

print("=== SQL 조인 결과 ===")
sql_join.show(10)

=== SQL 조인 결과 ===
+-----------+-----------+------+---------+--------+
|       name| department|salary|dept_head|location|
+-----------+-----------+------+---------+--------+
|Employee_10|    Finance|118953|      Eve|   Seoul|
| Employee_9|  Marketing|117189|  Charlie|   Seoul|
|Employee_26|  Marketing|114065|  Charlie|   Seoul|
|Employee_15|         HR|112409|    Diana|   Daegu|
|Employee_16|      Sales|111211|      Bob|   Busan|
| Employee_6|      Sales|109479|      Bob|   Busan|
|Employee_39|Engineering|107563|    Alice|   Seoul|
| Employee_5|    Finance|107121|      Eve|   Seoul|
| Employee_8|  Marketing|106557|  Charlie|   Seoul|
|Employee_17|         HR|105697|    Diana|   Daegu|
+-----------+-----------+------+---------+--------+
only showing top 10 rows



In [ ]:
# -----------------------------------------------------------------------------
# expr(): SQL 표현식을 DataFrame에서 사용
# -----------------------------------------------------------------------------

# expr(): SQL 표현식을 Column으로 변환
# select(), withColumn() 등에서 SQL 문법 사용 가능

# SQL 표현식으로 새 컬럼 추가
df_with_expr = df_emp.withColumn(
    "salary_grade",
    expr("CASE WHEN salary >= 80000 THEN 'High' ELSE 'Normal' END")
).withColumn(
    "bonus",
    expr("salary * 0.1")  # 급여의 10%
)

print("=== expr() 사용 예시 ===")
df_with_expr.select("name", "salary", "salary_grade", "bonus").show(5)

### 실습 6-1: 임시 뷰 등록
#### df_emp를 "emp"라는 이름의 임시 뷰로 등록하세요.

In [6]:
df_emp.createOrReplaceTempView("emp")

#### 실습 6-2: SQL 쿼리 실행
##### SQL로 employees 테이블에서 salary가 70000 이상인 직원의 name과 salary를 조회하세요

In [ ]:
sql_result = spark.sql('''
            SELECT
                name,
                salary
            FROM employees
            WHERE salary >= 70000
'''
)
sql_result.show(5)

+----------+------+
|      name|salary|
+----------+------+
|Employee_2| 88555|
|Employee_4| 75920|
|Employee_5|107121|
|Employee_6|109479|
|Employee_8|106557|
+----------+------+
only showing top 5 rows



#### 실습 6-3: SQL 집계
##### SQL로 부서별 평균 급여를 조회하세요.

In [13]:
sql_avg_result = spark.sql('''
            SELECT
                department,
                ROUND(AVG(salary), 2) as `평균급여`
            FROM employees 
            GROUP BY department
''')
sql_avg_result.show(5)

+-----------+--------+
| department|평균급여|
+-----------+--------+
|         HR|85255.23|
|  Marketing| 76457.5|
|      Sales| 70509.7|
|Engineering|79437.14|
|    Finance| 82034.8|
+-----------+--------+



#### 실습 6-4: expr 사용
##### expr()을 사용하여 salary의 5%를 "bonus" 컬럼으로 추가하세요.

In [15]:
df_emp.withColumn("bonus", expr("salary * 0.05")).select("name", "salary", "bonus").show(5)

+----------+------+-------+
|      name|salary|  bonus|
+----------+------+-------+
|Employee_1| 63483|3174.15|
|Employee_2| 88555|4427.75|
|Employee_3| 57159|2857.95|
|Employee_4| 75920|3796.00|
|Employee_5|107121|5356.05|
+----------+------+-------+
only showing top 5 rows

